In [7]:
import torch
import numpy as np
import os
import sys
import tqdm

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
device = torch.device("cuda")
# sys.path.append(work_item.attrib("cwd")[0])
# os.makedirs(work_item.attrib("output")[0], exist_ok=True)
ckpt = torch.load("01_density_only/our_high_res_000029.tar")

pinf_data_test = np.load("data/test_dataset_full.npz")
IMAGE_TEST_np = pinf_data_test['images_test']
POSES_TEST_np = pinf_data_test['poses_test']
HWF_np = pinf_data_test['hwf']
NEAR_float = pinf_data_test['near'].item()
FAR_float = pinf_data_test['far'].item()
H_int = int(HWF_np[0])
W_int = int(HWF_np[1])
FOCAL_float = float(HWF_np[2])
VOXEL_TRAN_np = pinf_data_test['voxel_tran']
VOXEL_SCALE_np = pinf_data_test['voxel_scale']
depth_size = 192

############################## Load BoundingBox ##############################
import src.bbox as bbox
voxel_tran_inv = np.linalg.inv(VOXEL_TRAN_np)
BBOX_MODEL_gpu = bbox.BBox_Tool(voxel_tran_inv, VOXEL_SCALE_np, device=device)
############################## Load BoundingBox ##############################

############################## Load Encoder ##############################
import src.encoder2 as encoder
ENCODER_gpu = encoder.HashEncoderNative(device=device).to(device)
ENCODER_gpu.load_state_dict(ckpt['embed_fn_state_dict'])
############################## Load Encoder ##############################

############################## Load Model ##############################
import src.model as model
MODEL_gpu = model.NeRFSmall(num_layers=2,
                            hidden_dim=64,
                            geo_feat_dim=15,
                            num_layers_color=2,
                            hidden_dim_color=16,
                            input_ch=ENCODER_gpu.num_levels * 2).to(device)
MODEL_gpu.load_state_dict(ckpt['network_fn_state_dict'])
############################## Load Model ##############################

rx, ry, rz = 128, 192, 128
xs, ys, zs = torch.meshgrid([torch.linspace(0, 1, rx, device=device), torch.linspace(0, 1, ry, device=device), torch.linspace(0, 1, rz, device=device)], indexing='ij')
coord_3d_sim = torch.stack([xs, ys, zs], dim=-1)  # [X, Y, Z, 3]
coord_3d_world = BBOX_MODEL_gpu.sim2world(coord_3d_sim)  # [X, Y, Z, 3]
N_timesteps = IMAGE_TEST_np.shape[0]
test_timesteps = torch.arange(N_timesteps, device=device, dtype=torch.float32) / (N_timesteps - 1)
for i in tqdm.trange(0, test_timesteps.shape[0]):
    test_timesteps_expended = test_timesteps[i].expand(coord_3d_world[..., :1].shape)
    POINTs_TIME_gpu = torch.cat([coord_3d_world, test_timesteps_expended], dim=-1)

C:\Users\imeho\AppData\Local\Temp\ipykernel_33848\3252385042.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("01_density_only/our_high_res_000029.tar"

tensor([ 0.0818, -0.0446, -0.0049,  0.0000], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.0084], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.0168], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.0252], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.0336], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.0420], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.0504], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.0588], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.0672], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.0756], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.0840], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.0924], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.1008], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.1092], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.1176], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.1261], device='cuda:0')
tensor([

100%|██████████| 120/120 [00:00<00:00, 269.88it/s]

tensor([ 0.0818, -0.0446, -0.0049,  0.8908], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.8992], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.9076], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.9160], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.9244], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.9328], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.9412], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.9496], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.9580], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.9664], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.9748], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.9832], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  0.9916], device='cuda:0')
tensor([ 0.0818, -0.0446, -0.0049,  1.0000], device='cuda:0')
